In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#PDFs in BDT and sindec?
import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
selection_version = 'version-001-p01'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    data_prefix = '/data/user/ssclafani/data/cscd/final'
    ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = '/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/followup/estes_sens_gp'
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository()
specs = cy.selections.ESTESDataSpecs.estes_10yr
specs_dnnc = cy.selections.DNNCascadeDataSpecs.DNNC_10yr


In [ ]:
%%time

ana_mc_bg = cy.get_analysis(
    repo, selection_version, specs,
    space_bg_kw={'bg_mc_weight':'bg_weight'},
    energy_kw={'bg_mc_weight': 'bg_weight'},
    load_sig=True,
)
ana = cy.get_analysis(
    cy.selections.Repository(), selection_version, specs, 
)

ana_combined = cy.get_analysis(
    cy.selections.Repository(), selection_version, specs + specs_dnnc, 
)

In [ ]:
a_both = ana_combined.anas[0]
a_mc_bg = ana_mc_bg.anas[0]
a = ana.anas[0]
a.sig, a_mc_bg.sig, a_both.sig

In [ ]:
a.bg_data, a_mc_bg.bg_data

## Helpers

In [ ]:
from cycler import cycle
from copy import deepcopy

soft_colors = cy.plotting.soft_colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


def get_bias_allt(tr, ntrials=200, n_sigs=np.r_[:101:10], quiet=False, poisson=False):
    trials = [
        (None if quiet else print(f'\r{n_sig:4d} ...', end='', flush=True))
        or
        tr.get_many_fits(ntrials, n_sig=n_sig, logging=False, seed=n_sig, poisson=poisson)
        for n_sig in n_sigs]
    if not quiet:
        print()
    for (n_sig, t) in zip(n_sigs, trials):
        t['ntrue'] = np.repeat(n_sig, len(t))
    allt = cy.utils.Arrays.concatenate(trials)
    return allt

def get_color_cycler():
    return cycle(colors)

def plot_ns_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)

    h = hl.hist((allt.ntrue, allt.ns), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(ax.set_ylim(lim))
    ax.plot(lim, lim, **expect_kw)
    ax.set_aspect('equal')

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$n_s$')
    ax.grid()

def plot_gamma_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)
    expect_gamma = tr.sig_injs[0].flux[0].gamma

    h = hl.hist((allt.ntrue, allt.gamma), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(lim)
    ax.set_ylim(1, 4)
    ax.axhline(expect_gamma, **expect_kw)

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$\gamma$')
    ax.grid()

def plot_bkg_trials(
            bg, fig=None, ax=None, 
            label='{} bg trials', 
            label_fit=r'$\chi^2[{:.2f}\mathrm{{dof}},\ \eta={:.3f}]$', 
            color=colors[0],
            density=False,
            bins=50,
        ):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if density:
        h = bg.get_hist(bins=bins).normalize()
    else:
        h = bg.get_hist(bins=bins)
    if label is not None:
        label = label.format(bg.n_total)
    hl.plot1d(ax, h, crosses=True, color=color, label=label)

    # compare with the chi2 fit:
    if hasattr(bg, 'pdf'):
        x = h.centers[0]
        norm = h.integrate().values
        if label_fit is not None:
            label_fit = label_fit.format(bg.ndof, bg.eta)
        if density:
            ax.semilogy(x, bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)
        else:
            ax.semilogy(x, norm * bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)

    ax.set_xlabel(r'TS')
    if density:
        ax.set_ylabel(r'Density')
    else:
        ax.set_ylabel(r'number of trials')
    ax.legend()
        
    return fig, ax

## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/estes_sens_test'

In [ ]:
def get_gp_tr(template_str, cutoff=np.inf, gamma=None, cpus=20, ana=ana, sigmas=None, sigsub=None, 
              update_bg=None, do_mc_injections=False):
    cutoff_GeV = cutoff * 1e3
    gp_conf = cg.get_gp_conf(
        template_str=template_str, gamma=gamma, cutoff_GeV=cutoff_GeV, base_dir=cg.base_dir)
    if sigmas is None:
        gp_conf['sigmas'] = np.logspace(np.log10(min(a.sig.sigma)), np.log10(max(a.sig.sigma)), 50+1)
    else:
        gp_conf['sigmas'] = sigmas
    if ana == ana_mc_bg:
        gp_conf['dir'] = os.path.join(gp_conf['dir'], 'mc_bg')
    elif ana == ana_combined:
        gp_conf['dir'] = os.path.join(gp_conf['dir'], 'combined')
    if sigsub is not None:
        gp_conf['sigsub'] = sigsub
        gp_conf['dir'] = os.path.join(gp_conf['dir'], 'sigsub_{}'.format(sigsub))
    if update_bg is not None:
        gp_conf['update_bg'] = update_bg
        gp_conf['dir'] = os.path.join(gp_conf['dir'], 'sigsub_{}_update_bg_{}'.format(sigsub, update_bg))
    if do_mc_injections:
        kwargs = {
            'inj_conf': {
                'bg_weight_names':['astro_weight', 'atmo_weight', 'muon_weight'],
                 'randomize' :['ra', 'dec'],
            },
        }
    else:
        kwargs = {}
        
        
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus, **kwargs)
    return tr

def get_catalog_tr(catalog, cutoff=np.inf, gamma=2.0, cpus=20, ana=ana):
    catalog_file = os.path.join(
        cg.catalog_dir, '{}_ESTES_12.pickle'.format(catalog))
    cat = np.load(catalog_file, allow_pickle=True)
    src = cy.utils.Sources(dec=cat['dec_deg'], ra=cat['ra_deg'], deg=True)
    cutoff_GeV = cutoff * 1e3

    conf = cg.get_ps_conf(src=src, gamma=gamma, cutoff_GeV=cutoff_GeV)
    tr = cy.get_trial_runner(ana=ana, conf=conf, mp_cpus=cpus)
    return tr



## Get Trial Runners

In [ ]:
stacking_catalog_gammas = {
    'snr': 2.69,
    'pwn': 2.95,
    'unid': 2.80,
}

sigmas_both = np.deg2rad(np.r_[0.3:3.01:.125, 3:5:0.5, 5:20, 20:40:2])


In [ ]:

tr_dict = {
    'pi0': get_gp_tr('pi0'),
    'kra5': get_gp_tr('kra5'),
    'kra50': get_gp_tr('kra50'),
    'snr': get_catalog_tr('snr', gamma=stacking_catalog_gammas['snr']),
    'pwn': get_catalog_tr('pwn', gamma=stacking_catalog_gammas['pwn']),
    'unid': get_catalog_tr('unid', gamma=stacking_catalog_gammas['unid']),
    'pi0_mc_bg': get_gp_tr('pi0', ana=ana_mc_bg),
    'kra5_mc_bg': get_gp_tr('kra5', ana=ana_mc_bg),
    'kra50_mc_bg': get_gp_tr('kra50', ana=ana_mc_bg),
    'snr_mc_bg': get_catalog_tr('snr', gamma=stacking_catalog_gammas['snr'], ana=ana_mc_bg),
    'pwn_mc_bg': get_catalog_tr('pwn', gamma=stacking_catalog_gammas['pwn'], ana=ana_mc_bg),
    'unid_mc_bg': get_catalog_tr('unid', gamma=stacking_catalog_gammas['unid'], ana=ana_mc_bg),
    'pi0_both': get_gp_tr('pi0', ana=ana_combined, sigmas=sigmas_both),
    #'kra5_both': get_gp_tr('kra5', ana=ana_combined, sigmas=sigmas_both),
    
    'pi0_nosigsub': get_gp_tr('pi0', sigsub=False),
    'pi0_mc_bg_nosigsub': get_gp_tr('pi0', ana=ana_mc_bg, sigsub=False),
    'pi0_mc_bg_nosigsub_v2': get_gp_tr('pi0', ana=ana_mc_bg, sigsub=False, update_bg=False, do_mc_injections=True),
}


## Test Bias

#### Kra-Gamma

In [ ]:
%%time

mp_cpus = 20

runners_to_check = [
   'kra5',
   'kra50', 'pi0',
]


fig, ax = plt.subplots(1, 1)
for label in runners_to_check:
    trial_runner = tr_dict[label]
    allt_i = get_bias_allt(
        trial_runner, ntrials=100, n_sigs=np.r_[:100:10])
    plot_ns_bias(ax, trial_runner, allt_i, label=label)
    
ax.set(title=r'GP Template Bias')
ax.legend(fontsize=8)
fig.tight_layout()
fig.savefig('{}/bias_gp_templates.png'.format(plot_dir))


#### Get Background trials

In [ ]:
bkg_file_dict = {
    'pi0': os.path.join(plot_dir, 'trials_{}.pkl'.format('pi0')),
    'kra5': os.path.join(plot_dir, 'trials_{}.pkl'.format('kra5')),
    'kra50': os.path.join(plot_dir, 'trials_{}.pkl'.format('kra50')),
    'snr': os.path.join(plot_dir, 'trials_{}.pkl'.format('snr')),
    'pwn': os.path.join(plot_dir, 'trials_{}.pkl'.format('pwn')),
    'unid': os.path.join(plot_dir, 'trials_{}.pkl'.format('unid')),
    
    'pi0_mc_bg': os.path.join(plot_dir, 'trials_{}.pkl'.format('pi0_mc_bg')),
    'kra5_mc_bg': os.path.join(plot_dir, 'trials_{}.pkl'.format('kra5_mc_bg')),
    'kra50_mc_bg': os.path.join(plot_dir, 'trials_{}.pkl'.format('kra50_mc_bg')),
    'snr_mc_bg': os.path.join(plot_dir, 'trials_{}.pkl'.format('snr_mc_bg')),
    'pwn_mc_bg': os.path.join(plot_dir, 'trials_{}.pkl'.format('pwn_mc_bg')),
    'unid_mc_bg': os.path.join(plot_dir, 'trials_{}.pkl'.format('unid_mc_bg')),
    
    'pi0_both': os.path.join(plot_dir, 'trials_{}.pkl'.format('pi0_both')),
    'kra5_both': os.path.join(plot_dir, 'trials_{}.pkl'.format('kra5_both')),
    
    'pi0_nosigsub': os.path.join(plot_dir, 'trials_{}.pkl'.format('pi0_nosigsub')),
    'pi0_mc_bg_nosigsub': os.path.join(plot_dir, 'trials_{}.pkl'.format('pi0_mc_bg_nosigsub')),
    'pi0_mc_bg_nosigsub_v2': os.path.join(plot_dir, 'trials_{}.pkl'.format('pi0_mc_bg_nosigsub_v2')),
}
n_bkg_trials = 20000
seed = 1337

bkg_dict = {}
for key, tr in tr_dict.items():
    if key in bkg_file_dict and os.path.exists(bkg_file_dict[key]):
        print('Loading background trials for template {}'.format(key))
        sig = np.load(bkg_file_dict[key], allow_pickle=True)
        bkg_dict[key] = sig.ts
    
    else:
        print('Running background trials for {}'.format(key))
        trials = tr.get_many_fits(
            n_trials=n_bkg_trials, seed=seed, mp_cpus=25)
        
        bkg_dict[key] = trials.ts
        
        out_file = os.path.join(plot_dir, 'trials_{}.pkl'.format(key))
        with open(out_file, 'wb') as f:
            pickle.dump(trials, f, protocol=2)
        

In [ ]:
from scipy import stats
stats.norm.sf(50)

tr_dict['pi0_mc_bg_nosigsub'].get_many_fits(10)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
bins = np.linspace(0, 20, 40)
c_cycler = cycle(colors)
for i, (name, bg) in enumerate(bkg_dict.items()):
    #if 'kra' in name or 'pi0' in name:
    if 'pi0' in name and not 'pi0_both' in name:
        color = next(c_cycler)
        bg = cy.dists.Chi2TSD(bg)
        plot_bkg_trials(bg, fig=fig, ax=ax, label=name, color=color, density=True, bins=bins)
        ax.axvline(bg.isf_nsigma(3), color=color, ls=':')
        #ax.axvline(bg.isf_nsigma(5), color=color, ls='-.')

ax.axvline(np.nan, color='0.7', ls=':', label=r'$3\sigma$')
ax.legend()
fig.tight_layout()
fig.savefig('{}/bkg_ts_dists.png'.format(plot_dir))

#### Test Sensitivity/Discovery Potential

In [ ]:
%%time

mp_cpus = 20
n_trials = 20000
seed = 592836
tol = 0.03

runners_to_check = [
   #'kra5',
   #'kra50', 
   'pi0_both',
   #'kra5_both',
]

results = {}
bg_trials = {}


for i, label in enumerate(runners_to_check):
    tr = tr_dict[label]

    bg = cy.dists.Chi2TSD(bkg_dict[label])
    print ('Finished bg trials')

    batch_size = 1000
    template_sens = tr.find_n_sig(
        bg.median(), 0.9, n_sig_step=10, seed=seed+4,
          batch_size = batch_size, tol=tol, mp_cups=mp_cpus)
    template_disc = tr.find_n_sig(
        bg.isf_nsigma(5), 0.5, n_sig_step=30, seed=seed+5,
        batch_size=batch_size, tol=tol, mp_cups=mp_cpus)

    if 'kra' in label:
        for t in [template_sens, template_disc]:
            t['model_norm'] = tr.to_model_norm(t['n_sig'])
            t['model_norm_upper'] = tr.to_model_norm(t['n_sig'] * (1+t['n_sig_error']))
            t['model_norm_lower'] = tr.to_model_norm(t['n_sig'] * (1-t['n_sig_error']))
    else:
        for t in [template_sens, template_disc]:
            t['E2dNdE'] = tr.to_E2dNdE(t['n_sig'], E0=100, unit=1000)
            t['E2dNdE_upper'] = tr.to_E2dNdE(t['n_sig'] * (1+t['n_sig_error']), E0=100, unit=1000)
            t['E2dNdE_lower'] = tr.to_E2dNdE(t['n_sig'] * (1-t['n_sig_error']), E0=100, unit=1000)


    results['{}_{:02d}_sens'.format(label, i)] = template_sens
    results['{}_{:02d}_disc'.format(label, i)] = template_disc
    bg_trials['{}_{:02d}'.format(label, i)] = bg



In [ ]:
print('Sensitivity:')
for name, values in results.items():
    i = int(name.split('_')[-2])
    if 'sens' in name:
        if 'kra' in name:
            print('  {:3.3f} [{:3.3f}, {:3.3f}] | {}'.format(
                values['model_norm'], 
                values['model_norm_lower'], 
                values['model_norm_upper'], 
                runners_to_check[i][1]))
        else:
            print('  {:3.2e} [{:3.2e}, {:3.2e}] | {}'.format(
                values['E2dNdE'], 
                values['E2dNdE_lower'], 
                values['E2dNdE_upper'], 
                runners_to_check[i][1]))

print('5sigma Discovery Potential:')
for name, values in results.items():
    i = int(name.split('_')[-2])
    if 'disc' in name:
        if 'kra' in name:
            print('  {:3.3f} [{:3.3f}, {:3.3f}] | {}'.format(
                values['model_norm'], 
                values['model_norm_lower'], 
                values['model_norm_upper'], 
                runners_to_check[i][1]))
        else:
            print('  {:3.2e} [{:3.2e}, {:3.2e}] | {}'.format(
                values['E2dNdE'], 
                values['E2dNdE_lower'], 
                values['E2dNdE_upper'], 
                runners_to_check[i][1]))
        

In [ ]:
def reweight(ow, energy, gamma, norm=1.0e-18, 
             e_pivot=1e5, energy_cutoff=None):
    """Reweight events according to power_law
    """
    n_types = 2.  # Dividing by n_types gives flux per flavor and per type
    weight = ow * norm * np.power(energy/e_pivot, -gamma) / n_types
    
    if energy_cutoff is not None:
        weight *= np.exp(-energy / energy_cutoff)
    return weight


fig, ax = plt.subplots()
bins = np.logspace(2, 8, 30)
weights = reweight(a.sig.oneweight, a.sig.true_energy, gamma=2.5) * a.livetime
ax.hist(a.bg_data.energy, bins=bins, histtype='step', label='Data', color='0.7', lw=3)
#ax.hist(a.sig.energy, bins=bins, weights=weights, histtype='step')
ax.hist(a.sig.energy, bins=bins, weights=a.sig.astro_weight * a.livetime, histtype='step', label='Astro')
ax.hist(a.sig.energy, bins=bins, weights=a.sig.atmo_weight * a.livetime, histtype='step', label='Atmo')
ax.hist(a.sig.energy, bins=bins, weights=a.sig.muon_weight * a.livetime, histtype='step', label='Muon')
ax.hist(a.sig.energy, bins=bins, weights=(a.sig.atmo_weight + a.sig.astro_weight + a.sig.muon_weight) * a.livetime, histtype='step', label='Total MC')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(1)
ax.legend()


In [ ]:
from scipy import stats
print(stats.chi2(2).isf(1 - .6826894921370859))
contour_fracs = [1 - 2*stats.norm.sf(1), 0.95]
dLLHs = stats.chi2.ppf (contour_fracs, 2)
dLLHs, 1 - 2*stats.norm.sf(1)

In [ ]:
k = 'pi0_mc_bg_nosigsub'
res = tr_dict[k].get_one_fit()
bg = cy.dists.Chi2TSD(bkg_dict[k])
fig, ax = plot_bkg_trials(bg)
fig.savefig('{}/trial_distribution_{}.png'.format(plot_dir, k))
print(bg.sf_nsigma(res[0]))
res, 

## Test Sources

In [ ]:
def model_norm_to_ns(tr, model_norm):
    # get a few values
    norm_100ns = trial_runner.to_model_norm(100)
    
    ns = model_norm / norm_100ns * 100
    assert np.allclose(model_norm, trial_runner.to_model_norm(ns))
    return ns

model_norm_to_ns(tr, model_norm=np.linspace(0.1, 1, 10))


In [ ]:
from cycler import cycle

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def get_color_cycler():
    return cycle(colors)


def test_kra_template(
            model_norm, 
            sigsub=True, cpus=15, 
            n_bg_trials=100000, 
            n_sig_trials=10000, 
        ):
    
    # get trial runner
    trial_runner = get_kra_tr(cpus=cpus, sigsub=sigsub)
    
    # compute expected number of events for given flux
    n_expected = model_norm_to_ns(tr=trial_runner, model_norm=model_norm)
    print('Number of expected signal events: {}'.format(n_expected))
    
    # get background trials
    print('Performing {} BG Trials'.format(n_bg_trials))
    bg = cy.dists.Chi2TSD(trial_runner.get_many_fits(n_trials=n_bg_trials, cpus=cpus))
    
    # get signal trials
    print('Performing {} Signal Trials'.format(n_sig_trials))
    sig_trials = trial_runner.get_many_fits(n_trials=n_sig_trials, cpus=cpus, n_sig=n_expected, poisson=True)
    
    # compute significance
    p_values = bg.sf(sig_trials.ts)
    n_sigma = bg.sf_nsigma(sig_trials.ts)
    
    return sig_trials, bg, p_values, n_sigma, n_expected
        

### Define Sources

In [ ]:
source_dict = {}
source_dict_no_ss = {}

for model_norm in np.linspace(0, 1, 11):
    source_dict[model_norm] = dict(model_norm=model_norm, sigsub=True)
    source_dict_no_ss[model_norm] = dict(model_norm=model_norm, sigsub=False)

print('Number of scan points:', len(source_dict), len(source_dict_no_ss))

### Run trials for each source

In [ ]:
n_bg_trials = 10000
n_sig_trials = 10000
cpus = 20


In [ ]:
results = {}
for name, cfg in source_dict.items():
    print('Running trials for: {}'.format(name))
    sig_tr_i, bg_i, p_values_i, n_sigma_i, n_expected_i = test_kra_template(
        model_norm=cfg['model_norm'],
        cpus=cpus, n_bg_trials=n_bg_trials, n_sig_trials=n_sig_trials,
        sigsub=cfg['sigsub'],
    )
    print()
    
    # save data
    results[name] = {
        'sig_trials': sig_tr_i,
        'bg_trials': bg_i,
        'p_values': p_values_i,
        'n_sigma': n_sigma_i,
        'n_expected': n_expected_i,
    }

    with open(os.path.join(plot_dir, 'trials.pkl'), 'wb') as f:
        pickle.dump(results, f, protocol=2)

In [ ]:
results_no_ss = {}
for name, cfg in source_dict_no_ss.items():
    print('Running trials for: {}'.format(name))
    sig_tr_i, bg_i, p_values_i, n_sigma_i, n_expected_i = test_kra_template(
        model_norm=cfg['model_norm'],
        cpus=cpus, n_bg_trials=n_bg_trials, n_sig_trials=n_sig_trials,
        sigsub=cfg['sigsub'],
    )
    print()
    
    # save data
    results_no_ss[name] = {
        'sig_trials': sig_tr_i,
        'bg_trials': bg_i,
        'p_values': p_values_i,
        'n_sigma': n_sigma_i,
        'n_expected': n_expected_i,
    }

    with open(os.path.join(plot_dir, 'trials_no_ss.pkl'), 'wb') as f:
        pickle.dump(results_no_ss, f, protocol=2)

In [ ]:
with open(os.path.join(plot_dir, 'trials.pkl'), 'rb') as f:
    results = pickle.load(f)
    
with open(os.path.join(plot_dir, 'trials_no_ss.pkl'), 'rb') as f:
    results_no_ss = pickle.load(f)

### Make Plots

In [ ]:

for name, res in results.items():
    res_no_ss = results_no_ss[name]
    print(name)
    
    fname = str(name)
    title = r'Model normalization: {:3.3f}'.format(name)
    
    mask = res['sig_trials'].ns != 0
    mask_bkg = res['bg_trials'].trials.ns != 0
    
    # -----------------
    # Significance Plot
    # -----------------
    fig, ax = plt.subplots(figsize=(6, 4))
    bins = np.linspace(-2, 6, 100)
    ax.hist(res['n_sigma'], bins=bins, histtype='step', color=colors[0])
    ax.hist(res_no_ss['n_sigma'], bins=bins, histtype='step', color=colors[1])
    ax.axvline(np.median(res['n_sigma']), ls='--', color=colors[0], label='Median: {:3.2f}$\cdot \sigma$'.format(np.median(res['n_sigma'])))
    ax.axvline(np.median(res_no_ss['n_sigma']), ls='--', color=colors[1], label='Median: {:3.2f}$\cdot \sigma$ [no sigsub]'.format(np.median(res_no_ss['n_sigma'])))
    ax.legend()
    ax.set_xlabel('$n \cdot \sigma$')
    ax.set_title(title)
    #ax.set_xscale('log')
    fig.tight_layout()
    fig.savefig(os.path.join(plot_dir, fname + '_significance.png'))
    
    # -------
    # TS Plot
    # -------
    fig, ax = plt.subplots(figsize=(6, 4))
    bins = np.linspace(-1, 50, 100)
    ax.hist(res['sig_trials'].ts, bins=bins, histtype='step', label='Signal', color=colors[0])
    ax.hist(res_no_ss['sig_trials'].ts, bins=bins, histtype='step', ls='--', label='Signal [no sigsub]', color=colors[0])
    ax.hist(res['bg_trials'].trials.ts, bins=bins, histtype='step', label='BKG', color=colors[1])
    ax.hist(res_no_ss['bg_trials'].trials.ts, bins=bins, ls='--', histtype='step', label='BKG [no sigsub]', color=colors[1])
    #ax.axvline(np.median(res['sig_trials'].ts), ls='--', color='0.6', label='Median')
    #ax.axvline(np.median(res['bg_trials'].trials.ts), ls='--', color='0.6')
    ax.legend()
    ax.set_xlabel('TS')
    ax.set_yscale('log')
    ax.set_title(title)
    fig.tight_layout()
    fig.savefig(os.path.join(plot_dir, fname + '_ts.png'))
    
    # -------
    # ns Plot
    # -------
    fig, ax = plt.subplots(figsize=(6, 4))
    bins = np.linspace(0, 500, 100)
    ax.hist(res['sig_trials'].ns, bins=bins, histtype='step', label='Signal', color=colors[0])
    ax.hist(res_no_ss['sig_trials'].ns, bins=bins, histtype='step', ls='--', label='Signal [no sigsub]', color=colors[0])
    ax.hist(res['bg_trials'].trials.ns, bins=bins, histtype='step',label='BKG', color=colors[1])
    ax.hist(res_no_ss['bg_trials'].trials.ns, bins=bins, histtype='step', ls='--', label='BKG [no sigsub]', color=colors[1])
    ax.set_xlabel('ns')
    ax.set_yscale('log')
    ax.set_title(title)
    ax.axvline(res['n_expected'], ls='--', color='0.6', label='Avg. injected ns {:3.1f}'.format(res['n_expected']))
    ax.legend()
    fig.tight_layout()
    fig.savefig(os.path.join(plot_dir, fname + '_ns.png'))

    print()

#### Make significance scan plot

In [ ]:
n_sigmas = np.linspace(0, 10, 100)

probs = np.empty((len(results), len(n_sigmas)))
model_norms = sorted(results.keys())

for i, (name, res) in enumerate(results.items()):
    for j, n_sigma in enumerate(n_sigmas):
        mask = res['n_sigma'] >= n_sigma
        probs[i, j] = np.sum(mask) / len(res['n_sigma'])

fig, ax = plt.subplots()
#im = ax.contour(model_norms, n_sigmas, probs.T, levels=[0.5, 0.9], colorbar='black')
contours = ax.contour(model_norms, n_sigmas, probs.T, levels=[0.16, 0.5, 0.84], colors='1.0', ls='--', lw=3)
plt.clabel(contours, inline=True, fontsize=8)
im = ax.contourf(model_norms, n_sigmas, probs.T, levels=np.linspace(0, 1, 11))
ax.set_xlabel('$\mathrm{KRA}_\gamma^5$ Model Normalization')
ax.set_ylabel('$N \cdot \sigma$')
ax.axhline(3, color='0.8', ls='--')
ax.axhline(5, color='0.8', ls='-.')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Probability')
ax.grid(color='0.8', alpha=0.1)

fig.savefig(os.path.join(plot_dir, 'kra_gamma_5pev_sigma_scan.png'))
